In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
file_path = '/Users/diana/Desktop/isear-test.xlsx'
custom_headers = ['Emotions', 'Text']
df = pd.read_excel(file_path, skiprows=1, header=None, names=custom_headers)

conda install openpyxl

In [3]:
df.head ()

,Emotions,Text
0,joy,My first feeling of falling in love. Coming a...
1,joy,When I saw that the mark I had obtained in an ...
2,joy,When I learnt that I had been admitted to the ...
3,anger,In a hurry to get to lectures and the car woul...
4,guilt,"Not attending school, when am not sick - but ..."


In [4]:
total_count = df['Emotions'].value_counts().sum()
print (total_count)

1150


In [5]:
df['Emotions'].value_counts()

Emotions
shame      182
fear       168
anger      166
disgust    166
sadness    161
guilt      155
joy        152
Name: count, dtype: int64

In [6]:
emotions_set = list (set (df['Emotions'].values))

In [63]:
#print (df['Text'].dtype)
df['Text'] = df['Text'].str.lower ()
#print (df['Text'])

text_column = df['Text']
tokenized_text = text_column.str.split()

for index, sentence in enumerate(tokenized_text):
    emotion_label = df['Emotions'][index]
    
    #print(emotion_label, sentence)

vocab = set() 
for tokens_list in tokenized_text:
    for token in tokens_list:
        if r'[^\w\s]' in token:
            if re.search(r'[^\w\s]', token):
                token = re.sub(r'[^\w\s]+$', '', token)
                if token not in vocab:
                    vocab.add (token[-1])
        if token not in vocab:
            vocab.add(token)
#print (list(vocab))


0       my first feeling of falling in love.  coming a...
1       when i saw that the mark i had obtained in an ...
2       when i learnt that i had been admitted to the ...
3       in a hurry to get to lectures and the car woul...
4       not attending school, when  am not sick - but ...
                              ...                        
1145     once i saw some children hunting a cat to death.
1146                       being insulted by my roommate.
1147    i had a habit of wetting my bed when i was you...
1148     when i was selected to the university of zambia.
1149    when i learnt that i had been accepted at the ...
Name: Text, Length: 1150, dtype: object
joy ['my', 'first', 'feeling', 'of', 'falling', 'in', 'love.', 'coming', 'across', 'a', 'person', 'who', 'made', 'me', 'feel', 'love,', 'happiness,', 'joy,', 'giving', 'me', 'a', 'new', 'and', 'beautiful', 'future,', 'who', 'made', 'me', 'dream.']
joy ['when', 'i', 'saw', 'that', 'the', 'mark', 'i', 'had', 'obtained', 'i

In [85]:
from math import log

collection = [token for instance in tokenized_text for token in instance]

#tf-idf

def calculate_tfidf(token, document):
    tf = document.count(token) / len(document)
    idf = log(1 + (len(tokenized_text) / collection.count(token)))
    tfidf = tf * idf
    return tfidf

tfidf_scores_list = []

for instance in tokenized_text:
        if token not in instance:
            tfidf_scores = {token: calculate_tfidf(token, instance) for token in set(instance)}
            #print (tfidf_scores)

{'first': 0.11382326413911331, 'across': 0.2052244907448603, 'beautiful': 0.18141246827600924, 'future,': 0.2430478071973063, 'joy,': 0.2430478071973063, 'me': 0.17720468186637317, 'new': 0.17155200401562432, 'who': 0.186094130715017, 'giving': 0.2052244907448603, 'falling': 0.19533431330855705, 'dream.': 0.2430478071973063, 'and': 0.03498808100570242, 'love.': 0.18766957622488942, 'my': 0.03458972029727068, 'love,': 0.2191761262308928, 'person': 0.11536959411523884, 'coming': 0.1576893547456502, 'a': 0.057601788665117594, 'feeling': 0.13708360402684971, 'feel': 0.12662318399301914, 'of': 0.043198723051564414, 'happiness,': 0.2430478071973063, 'made': 0.22051192116668192, 'in': 0.04435726736074292}
{'was': 0.06485152629677803, 'i': 0.06879567088379086, 'good': 0.2201343745049886, 'that': 0.09964418435980023, 'an': 0.14470022230957857, 'obtained': 0.3972567287934932, 'a': 0.05220162097776282, 'the': 0.05133342897512176, 'mark': 0.32881009875026673, 'saw': 0.2063046662521429, 'in': 0.080

In [ ]:
#conda install scikit-learn

Channels:
 - defaults
 - conda-forge
 - huggingface
 - pytorch
Platform: osx-64
Solving environment: done

## Package Plan ##

  environment location: /Users/diana/miniconda3/envs/newenv

  added / updated specs:
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    joblib-1.2.0               |  py310hecd8cb5_0         402 KB
    scikit-learn-1.3.0         |  py310h3ea8b11_1         7.6 MB
    scipy-1.12.0               |  py310ha516a68_0        21.9 MB
    threadpoolctl-2.2.0        |     pyh0d69192_0          16 KB
    ------------------------------------------------------------
                                           Total:        30.0 MB

The following NEW packages will be INSTALLED:

  joblib             pkgs/main/osx-64::joblib-1.2.0-py310hecd8cb5_0 
  scikit-learn       pkgs/main/osx-64::scikit-learn-1.3.0-py310h3ea8b11_1 
  scipy              pkgs/main/osx-64::s

In [ ]:
#import sklearn

In [70]:
"""from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the corpus to obtain the TF-IDF sparse matrix
tfidf_matrix_sparse = tfidf_vectorizer.fit_transform(sentence)

#tfidf_vector_first_sentence = tfidf_matrix_sparse[0]
#print("First", tfidf_vector_first_sentence)
print (tfidf_matrix_sparse)"""



  (0, 9)	1.0
  (2, 4)	1.0
  (3, 7)	1.0
  (5, 3)	1.0
  (6, 2)	1.0
  (7, 0)	1.0
  (8, 1)	1.0
  (9, 8)	1.0
  (10, 5)	1.0
  (11, 6)	1.0


In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

from torch.autograd import Variable

In [ ]:
class Emotions (nn.Module):

    def __init__(self, input_size, hidden_1, output_size):

        super (Emotions, self).__init__()
        self.fc1 = nn.Linear (input_size, hidden_1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear (hidden_1, output_size)
    def forward (self, x):
        out = self.fc1 (x)
        out = self.relu1 (out)
        out = self.fc2 (out)
        return out



In [ ]:
model = Emotions (1150, 50, 7)

In [ ]:
import torch.utils

batch_size = 50
train_loader = torch.utils.data.DataLoader (dataset = df, batch_size=batch_size, shuffle =True)